# WORD ANALYSIS

**INPUTS**:
* *data/01_CIE_emedicine_info.xlsx*
* *data/risk_texts.xlsx*
* *data/112/EM_112_ICD_selected.csv*
* **STOPWORDS**

**OUTPUTS**:
* *data/02_CIE_tokenized_info.xlsx*
* *data/02_112_patients_db.xlsx*

**NOTAS**

### Imports

In [1]:
import pandas as pd
import re #regex
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
#from googletrans import Translator
from deep_translator import GoogleTranslator

[nltk_data] Downloading package stopwords to /Users/nacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## CONFIGURATION

### Get data - Virtual Patients

In [ ]:
webs = pd.read_excel("data/01_CIE_emedicine_info.xlsx")
webs.fillna('')

risk_words = pd.read_excel("data/risk_texts.xlsx")

### Get data - 112 Database

In [ ]:
db_112 = pd.read_csv("data/112/EM_112_ICD_selected.csv")[['DIA_CODIGO','PAC_SEXO','LAB_RIESGO_VITAL','CALC_AGE']]
db_112.rename({'DIA_CODIGO': 'CIE', 'CALC_AGE': 'AGE', 'PAC_SEXO': 'SEX', 'LAB_RIESGO_VITAL': 'RISK'}, axis=1, inplace=True)

db_112_texts = pd.read_csv("data/112/EM_112_ICD_selected.csv")[['ARB_VARIABLES','VAR_Administración de medicación en el momento de la sintomatología','VAR_Alteraciones de la piel','VAR_Alteración de la conciencia','VAR_Alteración de la conducta','VAR_Antecedentes','VAR_Aparición manchas cutáneas','VAR_Apetito','VAR_Arritmia activa','VAR_Atención previa','VAR_Atragantamiento','VAR_Causalidad de la ingesta','VAR_Cianosis','VAR_Clínica vascular arterial (dolor, palidez y/o frialdad)','VAR_Clínica vascular venosa (calor, edema y enrojecimiento)','VAR_Congestión nasal','VAR_Consecuencias de la clínica','VAR_Consumo de sustancias tóxicas','VAR_Convulsiones','VAR_Crisis hipertensiva','VAR_Criterios código ICTUS','VAR_Criterios epidemiológicos','VAR_Cuadro Gastrointestinal (Náuseas, vómitos y/o diarrea)','VAR_Cuadro Vegetativo','VAR_Descompensación aguda de enfermedad mental','VAR_Desencadenantes de la clínica','VAR_Deshidratación','VAR_Deterioro nivel conciencia','VAR_Diarrea','VAR_Disnea','VAR_Disuria y/o Hematuria','VAR_Dolor','VAR_Edad','VAR_Embarazada','VAR_Enfermedad infecciosa epidemiológica','VAR_Estado de humor','VAR_Estreñimiento','VAR_Evolución clínica','VAR_Existencia de focalidad neurológica','VAR_Fallecimiento','VAR_Fiebre','VAR_Glucemia','VAR_Gravedad de la lesión/lesiones','VAR_Hemorragia','VAR_Inconsciente','VAR_Inconsciente recuperado','VAR_Ingesta de sustancia(Medicamento o Tóxicos)','VAR_Ingesta medicamentosa','VAR_Ingesta producto doméstico','VAR_Inicio de la clínica','VAR_Intento de suicidio activo','VAR_Lugar de hemorragia','VAR_Lugar del incidente','VAR_Mareo','VAR_Medicación habitual','VAR_Menstruación','VAR_Nivel de actividad','VAR_Nivel de relación y contacto','VAR_Náuseas','VAR_Número de heridos','VAR_Parto en curso','VAR_Picor','VAR_Quemadura','VAR_Respiración','VAR_Sangre o moco en heces','VAR_Signos de gravedad','VAR_Sin más información','VAR_Sustancia tóxica','VAR_Síndrome gripal','VAR_Síntomas de edema de glotis','VAR_Tiempo de evolución','VAR_Tipo alteración en la piel','VAR_Tipo de accidente','VAR_Tratamiento','VAR_Trauma previo','VAR_Vómitos','VAR_Vómitos alimenticios/biliosos','INC_OBSERVACIONES']]
db_112_texts = db_112_texts.fillna("")
db_112_texts = db_112_texts.astype(str)

db_112['TEXT_ES'] = db_112_texts.apply(''.join, axis=1)

#### In case it has already been processed:

In [ ]:
db_112 = pd.read_excel("data/02_112_patients_db.xlsx")
db_112 = db_112.astype(str)

## FUNCTIONS

In [2]:
def tokenize_this(text):
    return list(filter(None,re.split("\s+",re.sub("[,;.:\*\-–_<>/\]\[)(?!%$ºª°'“\"+=\d\s]+", " ",text.lower()))))

def tokenize_and_filter(text):
    if not pd.isna(text):
        return [w for w in tokenize_this(text) if w not in get_stop_words()]
    return []

count_total= 0
def tokenized_words_to_str(text):
    global count_total
    count_total = count_total + 1
    if count_total % 10000 == 0:
        print(count_total)
    return " ".join(map(str, tokenize_and_filter(text)))

def get_all_words_one_list(list_of_lists):
    list_of_words = []
    for text_list in list_of_lists:
        list_of_words.extend(text_list)
    return list_of_words

def get_stop_words():
    stop_words = stopwords.words('english')
    try:
        f = open("stop_words/words_to_add.txt", "r")
        stop_words.extend(f.read().split("\n"))
    finally: 
        return stop_words

def days_to_year(days):
    try:
        return int(float(days)/365)
    except:
        return -1

def es_to_en(texto):
    return GoogleTranslator(source='es', target='en').translate(texto)

def list_es_to_en(texts_es):
    texts_en = []
    for i in range(int(136160 / 20)):
        in_es = " \n ".join(texts_es[i*20:(i+1)*20])
        in_en = es_to_en(in_es).split("\n")
        texts_en.extend(in_en)
        print(f'{i}')
    return texts_en

## TOKENIZE DATA

### Tokenize - Virtual Patients words

In [ ]:
webs["T_HISTORY"] = webs["HISTORY"].apply(tokenize_and_filter)
webs["T_EXAMINATION"] = webs["EXAMINATION"].apply(tokenize_and_filter)
webs["T_GLOBAL"] = webs["GLOBAL"].apply(tokenize_and_filter)
webs["TOKENIZED"] = webs["T_HISTORY"] + webs["T_EXAMINATION"] + webs["T_GLOBAL"]

webs["RISKWORDS"] = risk_words["RISKTEXTS"].apply(tokenize_and_filter)

### Tokenize - 112 Database

In [ ]:
# 1) Cambiamos etiquetas de riesgo {2.0: 1, 1.0: 0}
db_112['RISK'] = db_112['RISK'].apply(lambda value : int(value - 1))

# 2) Actualizamos la edad a días
db_112['AGE'] = db_112['CALC_AGE'].apply(days_to_year)

# 3) Eliminamos la varibale Trauma previo__NO, puesto que la tokenizará en 3 distintas
db_112['TEXT_ES'] = db_112['TEXT_ES'].apply(lambda text : text.replace('Trauma previo__NO', ''))

# 4) Tokenizamos
db_112['TEXT_ES'] = db_112['TEXT_ES'].apply(tokenized_words_to_str)

In [ ]:
# 5) Traducimos al inglés
db_112['TEXT'] = list_es_to_en(db_112['TEXT_ES'].tolist())

# 6) Tokenizamos y filtramos stop words
db_112['TEXT'] = db_112['TEXT'].apply(tokenized_words_to_str)

## GET STATISTICS

### Data for Virtual Patients

In [ ]:
all_words = get_all_words_one_list(webs["TOKENIZED"])
print(f"After tokenization and filtering, there are {len(all_words)} words in total.")
print(f"For filtering, it has been used {len(get_stop_words())} words.")
print(f"Total unique words: {len(set(all_words))}.")
#print(pd.DataFrame(all_words).value_counts())

### Data cases of 112

In [ ]:
print(f"In the 112-Database there are {len(db_112)} cases.")

## STORE DATA

### Store - Virtual Patients

In [ ]:
webs.drop(columns=['WEB','HISTORY','EXAMINATION','GLOBAL','T_HISTORY', 'T_EXAMINATION', 'T_GLOBAL'], inplace=True)
webs.to_excel("./data/02_CIE_tokenized_info.xlsx", index=False)

### Store - 112 Patients

In [ ]:
db_112.to_excel("./data/02_112_patients_db.xlsx", index=False)